In [1]:
!apt-get install portaudio19-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 8 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 1s (197 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 129824 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.1

In [2]:
!pip install pyaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 1.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyaudio: filename=PyAudio-0.2.13-cp310-cp310-linux_x86_64.whl size=63863 sha256=64947264e63cc9e46612c69840f8eff2fbb09b9af160dea9af8ecaed074ca3d9
  Stored in directory: /root/.cache/pip/wheels/14/f1/c2/d102b4765a82c5a7bb273998dca7e4a53fc58e9a1a516fda81
Successfully built pyaudio


In [3]:
!pip install noisereduce

In [4]:
pip install soundfile

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pyaudio
import struct
import numpy as np
import tensorflow as tf
import librosa
import wave
from scipy.io import wavfile
import IPython.display as ipd
import noisereduce as nr



In [9]:
import cv2
import numpy as np
import soundfile as sf
import librosa
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define the emotions mapping
emotions = {1: 'neutral', 2: 'happy',3: 'calm' ,4: 'sad', 5: 'angry', 6: 'fearful', 7: 'disgust', 8: 'surprised'}

# Step 1: Extract Audio from Video
# Step 1: Extract Audio from Video
def extract_audio(video_path, output_path):
    video = cv2.VideoCapture(video_path)
    _, audio = video.read()
    audio = audio.mean(axis=1) if len(audio.shape) > 1 else audio
    sf.write(output_path, audio, 44100)  # Save the audio as a wave file
    return audio, output_path


# Step 2: Speech Activity Detection (Example using librosa)
def speech_activity_detection(audio):
    # Convert audio to mono if it is in stereo format
    if len(audio.shape) > 1:
        audio = np.mean(audio, axis=1)

    # Apply librosa's amplitude-based voice activity detection
    frames = librosa.effects.split(audio, top_db=20)

    # Get the segments of the audio that contain speech
    speech_segments = []
    for start, end in frames:
        speech_segments.append(audio[start:end])

    return speech_segments

# Step 3: Emotion Recognition
def emotion_recognition(audio_segments, sample_rate):
    # Load the pre-trained emotion recognition model
    model = load_model('/content/drive/MyDrive/Colab Notebooks/datasets/Speech recognition major module2.h5')

    predicted_emotions = []
    for segment in audio_segments:
        # Extract audio features from the speech segment
        features = extract_features(segment, sample_rate)

        # Preprocess the features (e.g., normalization, scaling)

        # Make predictions using the emotion recognition model
        prediction = model.predict(features)

        # Map the predicted output to emotion labels
        emotion_label = map_prediction_to_emotion(prediction)

        predicted_emotions.append(emotion_label)

    return predicted_emotions

# Helper function: Extract Audio Features
def extract_features(audio_segment, sample_rate):
    # Convert audio segment to mono if it is in stereo format
    if len(audio_segment.shape) > 1:
        audio_segment = audio_segment.mean(axis=1)

    # Extract Mel-frequency cepstral coefficients (MFCCs) as audio features
    mfccs = librosa.feature.mfcc(y=audio_segment, sr=sample_rate, n_mfcc=40)

    # Transpose and reshape MFCCs
    mfccs = mfccs.T.reshape(-1, 1)

    # Pad or truncate MFCCs to have a consistent number of frames
    max_frames = 40
    padded_mfccs = pad_sequences([mfccs], maxlen=max_frames, padding='post', truncating='post', dtype='float32')

    return padded_mfccs

# Helper function: Map Prediction to Emotion Labels
def map_prediction_to_emotion(prediction):
    # Map the model's prediction output to emotion labels based on the emotions mapping
    emotion_label = emotions[np.argmax(prediction)+1]

    return emotion_label

# Main Function
def predict_speech_emotion(video_path):
    # Step 1: Extract Audio from Video
    # Step 1: Extract Audio from Video
    wave_path = '/content/extracted_audio.wav'  # Path to save the extracted audio as a wave file
    audio, _ = extract_audio(video_path, wave_path)

    # Step 2: Speech Activity Detection
    speech_segments = speech_activity_detection(audio)

    # Step 3: Emotion Recognition
    sample_rate = 44100  # Define the sample rate of the audio data (adjust if necessary)
    predicted_emotions = emotion_recognition(speech_segments, sample_rate)

    return predicted_emotions

# Provide the path to the uploaded video
video_path = '/content/WhatsApp Video 2023-07-16 at 12.30.40 PM.mp4'

# Call the function to predict speech emotion
speech_result = predict_speech_emotion(video_path)

# Print the predicted emotions
print('Predicted Emotions:', speech_result)


/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=656
  warnings.warn(


1/1 [==============================] - 1s 804ms/step
Predicted Emotions: ['fearful']


In [10]:
import tensorflow as tf
import time
from keras.models import load_model
from keras.preprocessing import image
import cv2
import numpy as np
from tensorflow import keras
from collections import Counter
import collections

In [11]:
model =  tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/datasets/Mini_project_face_expression_model.h5")

In [12]:
# initialize the Haar Cascade face detection model
def Facerecognization():
    face_classifier = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
    classifier =load_model("/content/drive/MyDrive/Colab Notebooks/datasets/Mini_project_face_expression_model.h5")

    emotion_labels =[
        "angry",
        "fear",
        "happy",
        "neutral",
        "sad",
        "surprise",
        "disgust"
    ]

In [14]:
import cv2
import tensorflow as tf
import numpy as np
import collections

# Load the face cascade classifier
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the pre-trained emotion recognition model
model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/datasets/Mini_project_face_expression_model.h5')

# Define the emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

def detect_emotion(video_path):
    buffer_size = 30  # Number of frames to consider in the emotion buffer
    emotion_buffer = collections.deque(maxlen=buffer_size)  # Emotion buffer

    # Open the video file
    video_capture = cv2.VideoCapture(video_path)

    while True:
        ret, frame = video_capture.read()

        if not ret:
            break

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(frame_gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))

        for (x, y, w, h) in faces:
            face_roi_gray = frame_gray[y:y+h, x:x+w]
            face_roi_gray = cv2.resize(face_roi_gray, (48, 48))

            roi = face_roi_gray.astype('float') / 255.0
            roi = np.expand_dims(roi, axis=0)
            roi = np.expand_dims(roi, axis=-1)

            prediction = model.predict(roi)[0]
            max_prob = np.max(prediction)
            if max_prob > 0.5:  # Adjust the threshold as per your requirement
                label = emotion_labels[prediction.argmax()]
                emotion_buffer.append(label)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()

    if len(emotion_buffer) > 0:
        coll = collections.Counter(emotion_buffer)
        max_frequency = max(coll.values())
        most_frequent_emotions = [emotion for emotion, count in coll.items() if count == max_frequency]
        label = most_frequent_emotions[0]
    else:
        label = "No Emotion Detected"

    return label

# Provide the path to the uploaded video
video_path = '/content/WhatsApp Video 2023-07-16 at 12.30.40 PM.mp4'

# Call the function
face_result = detect_emotion(video_path)
print('Detected Emotion:', face_result)


1/1 [==============================] - 0s 60ms/step
Detected Emotion: Angry


In [15]:

speech_emotion_result= speech_result[0].lower() if speech_result else ""
face_emotion_result= face_result.lower()
def classify_conversation(face_emotion, speech_emotion):

    if (speech_emotion == "happy" and face_emotion == "surprise") or \
       (speech_emotion == "surprise" and face_emotion == "happy") or \
       (speech_emotion == "neutral" and face_emotion == "happy") or \
       (speech_emotion == "happy" and face_emotion == "neutral") or \
       (speech_emotion == "neutral" and face_emotion == "surprise") or \
       (speech_emotion == "surprise" and face_emotion == "neutral") or \
       (speech_emotion == "happy" and face_emotion == "happy") or \
       (speech_emotion == "surprise" and face_emotion == "surprise") or \
       (speech_emotion == "neutral" and face_emotion == "neutral"):
        return "pleasant"
    elif (speech_emotion == "angry" and face_emotion == "sad") or \
         (speech_emotion == "sad" and face_emotion == "angry") or \
         (speech_emotion == "sad" and face_emotion == "disgust") or \
         (speech_emotion == "disgust" and face_emotion == "sad") or \
         (speech_emotion == "disgust" and face_emotion == "angry") or \
         (speech_emotion == "angry" and face_emotion == "disgust"):
        return "argumentative"
    elif(speech_emotion == "angry" and face_emotion == "angry") or \
    (speech_emotion == "angry" and face_emotion == "fear")  or \
    (speech_emotion == "fearful" and face_emotion == "angry") :
        return "hustle"
    else:
        return "not determined"

face_emotion=face_emotion_result
speech_emotion=speech_emotion_result

conversation_category = classify_conversation(face_emotion, speech_emotion)
print("Conversation category:", conversation_category)


Conversation category: hustle
